# Data Preparation: Merging Datasets

Now that we have race, weather, and qualifying race data, we'll merge our sets together to create the dataset we'll feed to our Machine Learning algorithms.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import time
start = time.time()

In [3]:
import pandas as pd
import numpy as np

In [4]:
races = pd.read_csv('./data/races.csv')
results = pd.read_csv('./data/results.csv')
qualifying = pd.read_csv('./data/qualifying.csv')
driver_standings = pd.read_csv('./data/driver_standings.csv')
constructor_standings = pd.read_csv('./data/constructor_standings.csv')
weather = pd.read_csv('./data/weather.csv')

In [5]:
print(races.shape)
races.head()

(1057, 8)


season  round    circuit_id      lat      long      country        date  \
0    1950      1   silverstone  52.0786  -1.01694           UK  1950-05-13   
1    1950      2        monaco  43.7347   7.42056       Monaco  1950-05-21   
2    1950      3  indianapolis  39.7950 -86.23470          USA  1950-05-30   
3    1950      4    bremgarten  46.9589   7.40194  Switzerland  1950-06-04   
4    1950      5           spa  50.4372   5.97139      Belgium  1950-06-18   

                                                 url  
0  http://en.wikipedia.org/wiki/1950_British_Gran...  
1  http://en.wikipedia.org/wiki/1950_Monaco_Grand...  
2  http://en.wikipedia.org/wiki/1950_Indianapolis...  
3  http://en.wikipedia.org/wiki/1950_Swiss_Grand_...  
4  http://en.wikipedia.org/wiki/1950_Belgian_Gran...

In [6]:
print(results.shape)
results.head()

(24947, 13)


season  round   circuit_id       driver date_of_birth nationality  \
0    1950      1  silverstone       farina    1906-10-30     Italian   
1    1950      1  silverstone      fagioli    1898-06-09     Italian   
2    1950      1  silverstone  reg_parnell    1911-07-02     British   
3    1950      1  silverstone    cabantous    1904-10-08      French   
4    1950      1  silverstone       rosier    1905-11-05      French   

  constructor  grid       time    status  points  podium  \
0        alfa     1  8003600.0  Finished     9.0       1   
1        alfa     2  8006200.0  Finished     6.0       2   
2        alfa     4  8055600.0  Finished     4.0       3   
3        lago     6        NaN   +2 Laps     3.0       4   
4        lago     9        NaN   +2 Laps     2.0       5   

                                                 url  
0  http://en.wikipedia.org/wiki/1950_British_Gran...  
1  http://en.wikipedia.org/wiki/1950_British_Gran...  
2  http://en.wikipedia.org/wiki/1950_British_Gran...  
3  http://en.wikipedia.org/wiki/1950_British_Gran...  
4  http://en.wikipedia.org/wiki/1950_British_Gran...

In [7]:
print(qualifying.shape)
qualifying.rename(columns = {'grid_position': 'grid'}, inplace = True)
qualifying.head()

(435, 6)


grid            driver_name                    car qualifying_time  season  \
0     1   Max  Verstappen  VER  Red Bull Racing Honda        1:28.997    2021   
1     2   Lewis  Hamilton  HAM               Mercedes        1:29.385    2021   
2     3  Valtteri  Bottas  BOT               Mercedes        1:29.586    2021   
3     4  Charles  Leclerc  LEC                Ferrari        1:29.678    2021   
4     5     Pierre  Gasly  GAS       AlphaTauri Honda        1:29.809    2021   

   round  
0      1  
1      1  
2      1  
3      1  
4      1

In [8]:
driver_standings.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'] ,axis = 1, inplace = True)

print(driver_standings.shape)
driver_standings.head()

(27113, 6)


season  round       driver  driver_points  driver_wins  \
0    1950      1       farina            0.0          0.0   
1    1950      1      fagioli            0.0          0.0   
2    1950      1  reg_parnell            0.0          0.0   
3    1950      1    cabantous            0.0          0.0   
4    1950      1       rosier            0.0          0.0   

   driver_standings_pos  
0                   0.0  
1                   0.0  
2                   0.0  
3                   0.0  
4                   0.0

In [9]:
constructor_standings.drop(['constructor_points_after_race', 'constructor_wins_after_race','constructor_standings_pos_after_race' ],axis = 1, inplace = True)

print(constructor_standings.shape)
constructor_standings.head()

(12711, 6)


season  round constructor  constructor_points  constructor_wins  \
0    1958      1      cooper                 0.0               0.0   
1    1958      1     ferrari                 0.0               0.0   
2    1958      1    maserati                 0.0               0.0   
3    1958      2      cooper                 8.0               1.0   
4    1958      2     ferrari                 6.0               0.0   

   constructor_standings_pos  
0                        0.0  
1                        0.0  
2                        0.0  
3                        1.0  
4                        2.0

In [10]:
print(weather.shape)
weather.head()

(1057, 9)


season  round    circuit_id              weather  weather_warm  \
0    1950      1   silverstone    Sunny, mild, dry.             0   
1    1950      2        monaco            not found             0   
2    1950      3  indianapolis                Rainy             0   
3    1950      4    bremgarten  Warm, dry and sunny             1   
4    1950      5           spa  Warm, dry and sunny             1   

   weather_cold  weather_dry  weather_wet  weather_cloudy  
0             0            0            0               0  
1             0            0            0               0  
2             0            0            1               0  
3             0            1            0               0  
4             0            1            0               0

In [11]:
df1 = pd.merge(races, weather, how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country','weather'], axis = 1)
df2 = pd.merge(df1, results, how='inner', on=['season', 'round', 'circuit_id', 'url']).drop(['url','points', 'status', 'time'], axis = 1)

df3 = pd.merge(df2, driver_standings, how='left', on=['season', 'round', 'driver']) 
df4 = pd.merge(df3, constructor_standings, how='left', on=['season', 'round', 'constructor']) #from 1958

final_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_name', 'car'], axis = 1) #from 1983


In [12]:
final_df.head()

season  round circuit_id        date  weather_warm  weather_cold  \
0    2021      1    bahrain  2021-03-28             0             0   
1    2021      1    bahrain  2021-03-28             0             0   
2    2021      1    bahrain  2021-03-28             0             0   
3    2021      1    bahrain  2021-03-28             0             0   
4    2021      1    bahrain  2021-03-28             0             0   

   weather_dry  weather_wet  weather_cloudy          driver  ... constructor  \
0            0            0               0        hamilton  ...    mercedes   
1            0            0               0  max_verstappen  ...    red_bull   
2            0            0               0          bottas  ...    mercedes   
3            0            0               0          norris  ...     mclaren   
4            0            0               0         leclerc  ...     ferrari   

  grid podium  driver_points  driver_wins  driver_standings_pos  \
0    2      1            0.0          0.0                   0.0   
1    1      2            0.0          0.0                   0.0   
2    3      3            0.0          0.0                   0.0   
3    7      4            0.0          0.0                   0.0   
4    4      6            0.0          0.0                   0.0   

   constructor_points  constructor_wins  constructor_standings_pos  \
0                 0.0               0.0                        0.0   
1                 0.0               0.0                        0.0   
2                 0.0               0.0                        0.0   
3                 0.0               0.0                        0.0   
4                 0.0               0.0                        0.0   

   qualifying_time  
0         1:29.385  
1         1:28.997  
2         1:29.586  
3         1:29.974  
4         1:29.678  

[5 rows x 22 columns]

In [13]:
# calculate age of drivers

from dateutil.relativedelta import *

final_df['date'] = pd.to_datetime(final_df.date)
final_df['date_of_birth'] = pd.to_datetime(final_df.date_of_birth)
final_df['driver_age'] = final_df.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)
final_df.drop(['date', 'date_of_birth'], axis = 1, inplace = True)

In [14]:
# fill/drop nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    final_df[col].fillna(0, inplace = True)
    final_df[col] = final_df[col].map(lambda x: int(x))
    
final_df.dropna(inplace = True )

In [15]:
# convert to boolean

for col in ['weather_warm', 'weather_cold','weather_dry', 'weather_wet', 'weather_cloudy']:
    final_df[col] = final_df[col].map(lambda x: bool(x))


In [16]:
# calculate difference in qualifying times

try:
    final_df['qualifying_time'] = final_df.qualifying_time.map(lambda x: 0 if str(x) == '00.000' 
                             else(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])) if x != 0 else 0))
except:
    print(final_df['qualifying_time'])
final_df = final_df[final_df['qualifying_time'] != 0]
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round']).qualifying_time.diff()
final_df['qualifying_time'] = final_df.groupby(['season', 'round']).qualifying_time_diff.cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)


In [17]:
# get dummies

df_dum = pd.get_dummies(final_df, columns = ['circuit_id', 'nationality', 'constructor'] )

for col in df_dum.columns:
    if 'nationality' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'constructor' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'circuit_id' in col and df_dum[col].sum() < 70:
        df_dum.drop(col, axis = 1, inplace = True)
    
    else:
        pass

In [18]:
df_dum.shape

(357, 18)

In [19]:
df_dum.to_csv('./data/f1_df_final.csv', index = False)

In [20]:
df_dum.head()

season  round  weather_warm  weather_cold  weather_dry  weather_wet  \
1     2021      1         False         False        False        False   
0     2021      1         False         False        False        False   
2     2021      1         False         False        False        False   
4     2021      1         False         False        False        False   
15    2021      1         False         False        False        False   

    weather_cloudy          driver  grid  podium  driver_points  driver_wins  \
1            False  max_verstappen     1       2              0            0   
0            False        hamilton     2       1              0            0   
2            False          bottas     3       3              0            0   
4            False         leclerc     4       6              0            0   
15           False           gasly     5      17              0            0   

    driver_standings_pos  constructor_points  constructor_wins  \
1                      0                   0                 0   
0                      0                   0                 0   
2                      0                   0                 0   
4                      0                   0                 0   
15                     0                   0                 0   

    constructor_standings_pos  qualifying_time  driver_age  
1                           0            0.000          23  
0                           0            0.388          36  
2                           0            0.589          31  
4                           0            0.681          23  
15                          0            0.812          25

In [21]:
end = time.time()

In [22]:
import datetime
str(datetime.timedelta(seconds=(end - start)))

'0:00:01.105164'

In [23]:
print(str(end - start)+" seconds")

1.1051640510559082 seconds
